# Models Testing


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.activation import LeakyReLU

In [3]:
# Set seeds
torch.manual_seed(0)
np.random.seed(0)

In [4]:
synthetic_calls_path = '/content/drive/MyDrive/Progetto Stage/data/heston_mc_synthetic_calls_tot.csv'
synthetic_puts_path = '/content/drive/MyDrive/Progetto Stage/data/heston_mc_synthetic_puts_tot.csv'

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    
    return df

In [6]:
synthetic_calls = pd.read_csv(synthetic_calls_path, index_col=0)
synthetic_puts = pd.read_csv(synthetic_puts_path, index_col=0)

synthetic_calls = reduce_mem_usage(synthetic_calls)
synthetic_puts = reduce_mem_usage(synthetic_puts)

In [7]:
synthetic_options = pd.concat([synthetic_calls, synthetic_puts], axis=0)
synthetic_options = shuffle(synthetic_options, random_state=0)
synthetic_options = synthetic_options.reset_index()
synthetic_options = synthetic_options.drop('index', axis=1)

In [24]:
synthetic_options

,Price,Strike,Kappa,Rho,Theta,Xi,V_0,Interest Rate,Time to Expiration,Option Price,C,P
0,100,143.0,1.728516,0.158936,0.158936,0.158936,0.158936,0.058685,0.158936,0.000000,1,0
1,100,68.0,1.207031,0.547852,0.547852,0.547852,0.547852,0.073486,0.547852,0.957520,0,1
2,100,127.0,0.997070,0.672363,0.672363,0.672363,0.672363,0.014740,0.672363,26.546875,0,1
3,100,140.0,1.411133,0.494629,0.494629,0.494629,0.494629,0.052063,0.494629,38.750000,0,1
4,100,60.0,0.271729,0.102478,0.102478,0.102478,0.102478,0.035553,0.102478,38.593750,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
807995,100,71.0,0.553223,0.829590,0.829590,0.829590,0.829590,0.076904,0.829590,27.875000,1,0
807996,100,57.0,1.411133,0.825684,0.825684,0.825684,0.825684,0.084656,0.825684,42.437500,1,0
807997,100,135.0,1.419922,1.062500,1.062500,1.062500,1.062500,0.073792,1.062500,2.345703,1,0
807998,100,64.0,1.577148,1.038086,1.038086,1.038086,1.038086,0.091187,1.038086,0.202026,0,1


# Preprocessing

In [8]:
synthetic_options = pd.get_dummies(synthetic_options, prefix='', prefix_sep='')

In [9]:
input_sc = StandardScaler()
output_sc = StandardScaler()
input_data = input_sc.fit_transform(synthetic_options.drop('Option Price', axis=1))
output_data = output_sc.fit_transform(synthetic_options['Option Price'].values.reshape(-1, 1))

train_size = 0.8
val_size = 0.1

last_train_idx = int(np.round(len(input_data) * train_size))
last_val_idx = last_train_idx + int(np.round(len(input_data) * val_size))

X_train = input_data[0:last_train_idx]
X_val = input_data[last_train_idx:last_val_idx]
X_test = input_data[last_val_idx:]

y_train = output_data[0:last_train_idx]
y_val = output_data[last_train_idx:last_val_idx]
y_test = output_data[last_val_idx:]

In [10]:
X_train = Variable(torch.Tensor(X_train))
X_val = Variable(torch.Tensor(X_val))
X_test = Variable(torch.Tensor(X_test))

y_train = Variable(torch.Tensor(y_train))
y_val = Variable(torch.Tensor(y_val))
y_test = Variable(torch.Tensor(y_test))

# Model

In [11]:
CUDA = torch.cuda.is_available()
device = 'cuda:0' if CUDA else 'cpu'

In [12]:
class ResBlock(nn.Module):

  def __init__(self, module):
    super(ResBlock, self).__init__()
    self.module = module

  def forward(self, x):
    return self.module(x) + x

In [13]:
class HiddenLayer(nn.Module):

  def __init__(self, layer_size, act_fn):
      super(HiddenLayer, self).__init__()
      
      if act_fn == 'ReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ReLU())
      elif act_fn == 'LeakyReLU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.LeakyReLU())
      elif act_fn == 'ELU':
        self.layer = nn.Sequential(
          nn.Linear(layer_size, layer_size),
          nn.ELU())
    
  def forward(self, x):
    return self.layer(x)

In [14]:
class Net(nn.Module):

  def __init__(self, input_size, output_size, hidden_size, num_layers, act_fn):
    super(Net, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    if act_fn == 'ReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ReLU())
    elif act_fn == 'LeakyReLU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.LeakyReLU())
    elif act_fn == 'ELU':
      self.initial_layer = nn.Sequential(
          nn.Linear(self.input_size, self.hidden_size),
          nn.ELU())

    self.hidden_layers_list = []

    for i in range(num_layers // 2):
      self.hidden_layers_list.append(
          ResBlock(
            nn.Sequential(
                HiddenLayer(self.hidden_size, act_fn),
                HiddenLayer(self.hidden_size, act_fn)
            )
        )
      )

    self.hidden_layers = nn.Sequential(*self.hidden_layers_list)

    self.net = nn.Sequential(
        self.initial_layer,
        self.hidden_layers,
        nn.Linear(self.hidden_size, self.output_size)
    )
  
  def forward(self, x):
    return self.net(x)

In [15]:
def init_weights(m, init_m: str):

  @torch.no_grad()
  def init_uniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_normal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.normal_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xuniform(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.01)

  @torch.no_grad()
  def init_xnormal(m):
    if isinstance(m, nn.Linear):
      torch.nn.init.xavier_normal_(m.weight)
      m.bias.data.fill_(0.01)

  if init_m == 'uniform':
    m.apply(init_uniform)
  elif init_m == 'normal':
    m.apply(init_normal)
  elif init_m == 'xaiver uniform':
    m.apply(init_xuniform)
  elif init_m == 'xavier normal':
    m.apply(init_xnormal)

# Best models from cross validation

In [16]:
best_models = [
               {'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'ReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 4, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 6, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'},
               {'n_hidden': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_method': 'xavier uniform'}
]

# Training

In [25]:
input_size = 11
output_size = 1
batch_size = 1024
epochs = 125
lr = 1e-4

loss_fn = nn.MSELoss()

In [26]:
class OptDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [27]:
def evaluate(model, loss_fn, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = loss_fn(out, y.to(device))
    return loss.item()

In [28]:
def mape_loss(y_pred, y):
  return sum([np.abs(y_i - hy_i) / y_i for y_i, hy_i in zip(y_pred, y)]) / len(y_pred)

In [29]:
def get_mape_loss(model, X, y):
  model.eval()
  with torch.no_grad():
    out = model(X.to(device))
    loss = mape_loss(
        out.squeeze().cpu().detach().numpy(), 
        y.squeeze().cpu().detach().numpy())
    return loss

In [30]:
def test_models(
  models_dict,
  epochs,
  batch_size,
  X_train,
  y_train,
  X_val,
  y_val,
  X_test,
  y_test,
  loss_fn
):
  testing_results = pd.DataFrame(columns=
                               ['hidden_size',
                                'n_layers',
                                'act_fun',
                                'init_methods',
                                'mean_val_result',
                                'std_val_result',
                                'test_mse',
                                'test_mae',
                                'test_rmse',
                                'test_mape'])

  for model_dict in models_dict:
    model = Net(input_size,
                output_size, 
                model_dict['n_hidden'], 
                model_dict['n_layers'], 
                model_dict['act_fun']).to(device)
    init_weights(model, model_dict['init_method'])
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    validation_losses = []
    test_res = {
        'hidden_size': model_dict['n_hidden'],
        'n_layers': model_dict['n_layers'],
        'act_fun': model_dict['act_fun'],
        'init_methods': model_dict['init_method']
    }

    print('Model: ', test_res)

    for epoch in range(epochs):
      model.train()
      total_loss = 0
      start_time = time.time()
      i = 0

      for batch, batch_labels in DataLoader(OptDataset(X_train, y_train), batch_size):
        out = model(batch.to(device))
        optimizer.zero_grad()

        loss = loss_fn(out, batch_labels.to(device))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i > 0 and i % 25 == 0:
          avg_loss = total_loss / 50
          elapsed = time.time() - start_time
          print('| Epoch {:3d} | {:5d}/{:5d} batches | lr {:2.5f} | ms/batch {:5.2f} | '
                  'loss {:5.8f}'.format(
              epoch, i, len(X_train) // batch_size+1, lr, elapsed * 1000 / 50,
              avg_loss))
          start_time = time.time()
          total_loss = 0
        
        i += 1

      validation_losses.append(evaluate(model, loss_fn, X_val, y_val))
    
    mse_test = evaluate(model, loss_fn, X_test, y_test)
    mae_loss = nn.L1Loss()
    mae_test = evaluate(model, mae_loss, X_test, y_test)
    mape_test = get_mape_loss(model, X_test, y_test)
    validation_losses = np.array(validation_losses)
    test_res['mean_val_result'] = validation_losses.mean()
    test_res['std_val_result'] = validation_losses.std()
    test_res['test_mse'] = mse_test
    test_res['test_mae'] = mae_test
    test_res['test_rmse'] = np.sqrt(mse_test)
    test_res['test_mape'] = mape_test

    testing_results = testing_results.append(test_res, ignore_index=True)

  return testing_results

In [ ]:
testing_results = test_models(best_models,
            epochs,
            batch_size,
             X_train,
            y_train,
            X_val,
            y_val,
            X_test,
            y_test,
            loss_fn)

Model:  {'hidden_size': 400, 'n_layers': 8, 'act_fun': 'LeakyReLU', 'init_methods': 'xavier uniform'}
| Epoch   0 |    25/  632 batches | lr 0.00010 | ms/batch 11.60 | loss 0.27984494
| Epoch   0 |    50/  632 batches | lr 0.00010 | ms/batch  8.35 | loss 0.02398693
| Epoch   0 |    75/  632 batches | lr 0.00010 | ms/batch  7.37 | loss 0.00737579
| Epoch   0 |   100/  632 batches | lr 0.00010 | ms/batch  7.59 | loss 0.00463546
| Epoch   0 |   125/  632 batches | lr 0.00010 | ms/batch  7.87 | loss 0.00368702
| Epoch   0 |   150/  632 batches | lr 0.00010 | ms/batch  6.64 | loss 0.00321917
| Epoch   0 |   175/  632 batches | lr 0.00010 | ms/batch  7.75 | loss 0.00299266
| Epoch   0 |   200/  632 batches | lr 0.00010 | ms/batch  7.70 | loss 0.00276515
| Epoch   0 |   225/  632 batches | lr 0.00010 | ms/batch  7.46 | loss 0.00264495
| Epoch   0 |   250/  632 batches | lr 0.00010 | ms/batch  6.40 | loss 0.00270261
| Epoch   0 |   275/  632 batches | lr 0.00010 | ms/batch  7.93 | loss 0.00256

In [ ]:
testing_results

In [ ]:
testing_results.to_csv('/content/drive/MyDrive/Progetto Stage/results/testing_results_heston.csv')